In [1]:
from pandas import read_csv

all_data = read_csv('lmrd_review.csv', delimiter=',', error_bad_lines=False)

C:\Users\Edward\AppData\Local\Temp\ipykernel_26116\3238638070.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  all_data = read_csv('lmrd_review.csv', delimiter=',', error_bad_lines=False)


In [2]:
all_data.head()

,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [3]:
all_data['review'] = all_data['review'].str.replace('\n', ' ')
all_data['review'] = all_data['review'].str.replace('\r', ' ')
all_data['review'] = all_data['review'].str.replace('\t', ' ')
all_data['review'] = all_data['review'].str.replace('  ', ' ')

In [4]:
all_data.dropna(subset=['review'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier

# 데이터 분리
X = all_data["review"]
y = all_data["label"]

# 데이터셋을 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 벡터화
vectorizer = CountVectorizer(stop_words="english", max_features=5000)  # 상위 5000개의 단어만 사용
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 모델 정의
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "SGD": SGDClassifier(loss="log", random_state=42)
}

results = {}

# 모델 학습 및 평가
for model_name, model in models.items():
    print(f"\n[ {model_name} ]")
    
    # XGBoost 모델의 경우 입력 데이터를 np.float32로 변환
    if model_name == "XGBoost":
        X_train_vec = X_train_vec.astype(np.float32)
        X_test_vec = X_test_vec.astype(np.float32)
    
    # 모델 학습
    model.fit(X_train_vec, y_train)
    
    # 테스트 데이터로 예측
    y_pred = model.predict(X_test_vec)
    
    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    
    # 평가 결과 출력
    print(f"정확도: {accuracy:.2f}")
    print("분류 리포트:")
    print(classification_report(y_test, y_pred))

# 결과 비교
print("\n모델 별 정확도 비교:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.2f}")

c:\Users\Edward\anaconda3\envs\py\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")



[ Logistic Regression ]
정확도: 0.87
분류 리포트:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4945
           1       0.87      0.87      0.87      5055

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000


[ Naive Bayes ]
정확도: 0.85
분류 리포트:
              precision    recall  f1-score   support

           0       0.84      0.85      0.85      4945
           1       0.85      0.84      0.85      5055

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000


[ Random Forest ]
정확도: 0.85
분류 리포트:
              precision    recall  f1-score   support

           0       0.83      0.86      0.85      4945
           1       0.86      0.83      0.84      5055

    accuracy                           0.85     10000
   macro avg 

c:\Users\Edward\anaconda3\envs\py\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


정확도: 0.88
분류 리포트:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      4945
           1       0.89      0.86      0.88      5055

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000


모델 별 정확도 비교:
Logistic Regression: 0.87
Naive Bayes: 0.85
Random Forest: 0.85
XGBoost: 0.86
LightGBM: 0.86
SGD: 0.88


In [6]:
import numpy as np

# CountVectorizer에서 단어 목록 가져오기
feature_names = vectorizer.get_feature_names_out()

# 모델 별 긍정/부정 단어 상위 20개 추출 함수
def extract_top_words(model, feature_names, model_name):
    if hasattr(model, "coef_"):  # 선형 모델 (Logistic Regression, Linear SVM)
        coefficients = model.coef_[0]
        sorted_indices = np.argsort(coefficients)
        negative_top20 = [(feature_names[i], coefficients[i]) for i in sorted_indices[:20]]
        positive_top20 = [(feature_names[i], coefficients[i]) for i in sorted_indices[-20:]]
    elif hasattr(model, "feature_importances_"):  # 트리 기반 모델 (Random Forest)
        importances = model.feature_importances_
        sorted_indices = np.argsort(importances)
        negative_top20 = [(feature_names[i], importances[i]) for i in sorted_indices[:20]]
        positive_top20 = [(feature_names[i], importances[i]) for i in sorted_indices[-20:]]
    else:
        print(f"{model_name} 모델은 가중치 또는 중요도를 제공하지 않음.")
        return None, None

    return negative_top20, positive_top20

# 각 모델에 대해 단어 추출
for model_name, model in models.items():
    print(f"\n[ {model_name} ]")
    negative_top20, positive_top20 = extract_top_words(model, feature_names, model_name)
    
    if negative_top20 and positive_top20:
        print("\n부정 단어 상위 20개:")
        for word, weight in negative_top20:
            print(f"{word}: {weight:.4f}")
        
        print("\n긍정 단어 상위 20개:")
        for word, weight in positive_top20:
            print(f"{word}: {weight:.4f}")



[ Logistic Regression ]

부정 단어 상위 20개:
forgettable: -2.2855
waste: -2.2327
tripe: -2.2044
mst3k: -2.2024
uninspired: -2.1701
stinker: -2.0798
disappointment: -2.0553
poorly: -1.9404
uninteresting: -1.8908
wasting: -1.8608
obnoxious: -1.8185
mildly: -1.7950
worst: -1.7465
yawn: -1.7411
rourke: -1.7348
laughable: -1.7317
tedious: -1.6931
manipulative: -1.6416
dreadful: -1.6185
appalling: -1.5889

긍정 단어 상위 20개:
pitch: 1.2928
suitable: 1.2988
disappoint: 1.2989
complaint: 1.3027
complain: 1.3192
funniest: 1.3344
chilling: 1.3562
delightful: 1.3615
likewise: 1.3760
superbly: 1.3846
damned: 1.4089
superb: 1.4312
astonishing: 1.4416
underrated: 1.5009
fears: 1.5075
polished: 1.5510
segments: 1.5718
notch: 1.5753
finest: 1.6016
refreshing: 1.9081

[ Naive Bayes ]
Naive Bayes 모델은 가중치 또는 중요도를 제공하지 않음.

[ Random Forest ]

부정 단어 상위 20개:
ponyo: 0.0000
custer: 0.0000
hartley: 0.0000
venice: 0.0000
miyazaki: 0.0000
chess: 0.0000
abusive: 0.0000
widmark: 0.0000
cannibal: 0.0000
macbeth: 0.0000
witche

In [9]:
from nltk.corpus import opinion_lexicon
from nltk import download

# 감성 사전 다운로드
download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Edward\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [10]:
import re

# 긍정/부정 단어 리스트 로드
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

# 텍스트 전처리 및 감성 단어 카운트 함수
def preprocess_and_count_sentiment(text):
    # 텍스트 소문자로 변환
    text = text.lower()
    # 특수 문자 및 숫자 제거
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()

    # 긍정/부정 단어 카운트
    pos_count = sum(1 for word in words if word in positive_words)
    neg_count = sum(1 for word in words if word in negative_words)
    return text, pos_count, neg_count

In [11]:
# 데이터 전처리 및 감성 단어 카운트 적용
all_data["review_cleaned"], all_data["pos_count"], all_data["neg_count"] = zip(
    *all_data["review"].apply(preprocess_and_count_sentiment)
)

# 데이터 확인
all_data.head()

,review,label,review_cleaned,pos_count,neg_count
0,Bromwell High is a cartoon comedy. It ran at t...,1,bromwell high is a cartoon comedy it ran at th...,5,5
1,Homelessness (or Houselessness as George Carli...,1,homelessness or houselessness as george carlin...,17,14
2,Brilliant over-acting by Lesley Ann Warren. Be...,1,brilliant overacting by lesley ann warren best...,9,7
3,This is easily the most underrated film inn th...,1,this is easily the most underrated film inn th...,8,6
4,This is not the typical Mel Brooks film. It wa...,1,this is not the typical mel brooks film it was...,6,2


In [13]:
from scipy.sparse import hstack

# 기존 텍스트 벡터화
X_text_vec = vectorizer.fit_transform(all_data["review_cleaned"])

# 긍정/부정 단어 카운트 추가
X_features = hstack([X_text_vec, all_data[["pos_count", "neg_count"]].values])

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_features, all_data["label"], test_size=0.2, random_state=42)


# 모델 학습 및 평가
for model_name, model in models.items():
    print(f"\n[ {model_name} ]")
    
    # XGBoost 모델의 경우 입력 데이터를 np.float32로 변환
    if model_name == "XGBoost":
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)
    
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 테스트 데이터로 예측
    y_pred = model.predict(X_test)
    
    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    
    # 평가 결과 출력
    print(f"정확도: {accuracy:.2f}")
    print("분류 리포트:")
    print(classification_report(y_test, y_pred))

# 결과 비교
print("\n모델 별 정확도 비교:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.2f}")


[ Logistic Regression ]
정확도: 0.87
분류 리포트:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4945
           1       0.87      0.87      0.87      5055

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000


[ Naive Bayes ]
정확도: 0.86
분류 리포트:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      4945
           1       0.86      0.85      0.86      5055

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000


[ Random Forest ]
정확도: 0.85
분류 리포트:
              precision    recall  f1-score   support

           0       0.84      0.85      0.85      4945
           1       0.85      0.84      0.85      5055

    accuracy                           0.85     10000
   macro avg 

c:\Users\Edward\anaconda3\envs\py\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


정확도: 0.87
분류 리포트:
              precision    recall  f1-score   support

           0       0.90      0.83      0.87      4945
           1       0.85      0.91      0.88      5055

    accuracy                           0.87     10000
   macro avg       0.88      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000


모델 별 정확도 비교:
Logistic Regression: 0.87
Naive Bayes: 0.86
Random Forest: 0.85
XGBoost: 0.86
LightGBM: 0.86
SGD: 0.87


빈도 기반 모델 별 정확도 비교:
Logistic Regression: 0.87
Naive Bayes: 0.85
Random Forest: 0.85
XGBoost: 0.86
LightGBM: 0.86
SGD: 0.88